# Import

## Library

In [ ]:
# Basic libraries
import numpy as np
import pandas as pd
import warnings

# Plotting and visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from scipy.stats import probplot
import statsmodels.formula.api as smf

# Set global font size for Seaborn
sns.set(font_scale=1.4)  # This will scale the fonts by a factor of 1.4
# Set the default font size to 18 for all text elements
plt.rcParams.update({'font.size': 18})

# PyTorch and deep learning
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from d2l import torch as d2l

# Preprocessing and feature engineering
from math import radians, sin, cos, sqrt, atan2
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import category_encoders as ce
import pylab

# Clustering
from sklearn.cluster import DBSCAN, KMeans

# Modeling and evaluation
from sklearn import metrics
from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV, LassoCV, Lasso, ElasticNet, Ridge
from sklearn.model_selection import train_test_split, RandomizedSearchCV, ParameterGrid, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
from xgboost import plot_importance
import statsmodels.formula.api as smf

# Miscellaneous
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
%load_ext autoreload
%autoreload 2


from encoders import *
from featureSelection import *

## Data

In [ ]:
# main data
train_data_path = './dataset/train.csv'
test_data_path = './dataset/test.csv'

# auxiliary data
mrt_exist_data_path = './dataset/auxiliary-data/sg-mrt-existing-stations.csv'
mrt_planned_data_path = './dataset/auxiliary-data/sg-mrt-planned-stations.csv'
mall_data_path = './dataset/auxiliary-data/sg-shopping-malls.csv'
primary_school_data_path = './dataset/auxiliary-data/sg-primary-schools.csv'
coe_price_data_path = './dataset/auxiliary-data/sg-coe-prices.csv'
stock_price_data_path = './dataset/auxiliary-data/sg-stock-prices.csv'

# Load Data
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
test_data['monthly_rent'] = 0
mrt_exist_data = pd.read_csv(mrt_exist_data_path)
mrt_planned_data = pd.read_csv(mrt_planned_data_path)
mall_data = pd.read_csv(mall_data_path)
primary_school_data = pd.read_csv(primary_school_data_path)
coe_price_data = pd.read_csv(coe_price_data_path)
stock_price_data = pd.read_csv(stock_price_data_path)

# 0. Cleaning

1. 新加坡的住宅物业: 二房式灵活组屋（2-room flexi flat）、三房式组屋（3-room flat）、四房式组屋（4-room flat）、五房式组屋（5-room flat）、三代同堂组屋（3Gen flat）、公寓式组屋（Executive flat）

In [ ]:
# flat_type
train_data['flat_type'] = train_data['flat_type'].str.replace("-", " ")
test_data['flat_type'] = test_data['flat_type'].str.replace("-", " ")

# mrt_planned
mrt_planned_data['opening_year'].replace('TBA', pd.NA, inplace=True)
mrt_planned_data.dropna(subset=['opening_year'], inplace=True)
# only include those planned within 3 years, due to the policy restriction
mrt_planned_data = mrt_planned_data[(mrt_planned_data['opening_year']).astype(int) < 2029]

train_data.head()
print(train_data.shape)

# 1. EDA - Main Data


## 1.1 General Info

In [ ]:
train_data.info()

## 1.2 Descriptive Statistics

In [ ]:
train_data.describe()

## 1.3 Visualization of General Info

In [ ]:
sns.pairplot(train_data,height=2)
plt.show()

#### 1.4 b) Feature Correlations

In [ ]:
# # Compute the correlation matrix
# corr_matrix = train_data.corr()
#
# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Feature Correlation Heatmap')
# plt.show()

### Observations

There is no null value for any column in the dataset.
The scale of numerical data varies significantly, remember to normalize before analysis
There are natural groups but not clear, further observation required

## 1.4 Univariate Analysis - Categorical

#### 1.4 a) town & subarea

In [ ]:
town_counts = train_data['town'].value_counts()
planning_area_counts = train_data['planning_area'].value_counts()

# Merge the two Series into a single DataFrame
combined_df = pd.DataFrame({'Town': town_counts, 'Planning Area': planning_area_counts}).fillna(0)

# Plotting
plt.figure(figsize=(20, 10))

# Width of a bar
width = 0.4

# Positions of bars on x axis
r1 = range(len(combined_df))
r2 = [x + width for x in r1]

# Creating bars
plt.bar(r1, combined_df['Town'], width=width, label='Town', color='blue', edgecolor='grey')
plt.bar(r2, combined_df['Planning Area'], width=width, label='Planning Area', color='red', edgecolor='grey')

# Title & Subtitle
plt.title('Comparing Town and Planning Area Counts', fontweight='bold')

# X axis
plt.xlabel('Areas', fontweight='bold')
plt.xticks([r + width for r in range(len(combined_df))], combined_df.index, rotation=90)

# Y axis
plt.ylabel('Counts', fontweight='bold')

# Show the legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()

#### 1.4 b) flat_type

In [ ]:
train_data['flat_type'].value_counts()

#### 1.4 c) flat_model

In [ ]:
flat_model_counts = train_data['flat_model'].value_counts()

# Plotting
plt.figure(figsize=(15,6))  # Adjust the figure size
sns.barplot(y=flat_model_counts.index, x=flat_model_counts.values, palette='viridis')  # Use y for town names to get a horizontal bar plot

# Add titles and labels
plt.title('Number of Occurrences by Flat Model')
plt.xlabel('Number of Occurrences')
plt.ylabel('Flat Model')

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
train_data['flat_model'].unique()

#### 1.4 d) furnished

In [ ]:
train_data['furnished'].value_counts()

#### 1.4 e) latitude & longitude


In [ ]:
# latitude = train_data['latitude']
# longitude = train_data['longitude']
#
# avg_lat, avg_lon = latitude.mean(), longitude.mean()
#
# # Create a base map
# m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)
#
# # Add the heat map
# heat_data = [[lat, lon] for lat, lon in zip(train_data['latitude'], train_data['longitude'])]
# HeatMap(heat_data).add_to(m)
#
# # Display the map if NEEDED
# # m

#### 1.4 f) subzone

In [ ]:
train_data['subzone'].value_counts()

#### 1.4 g) region

In [ ]:
train_data['region'].value_counts()

#### 1.4 h) block

In [ ]:
train_data['block'].value_counts()

## 1.5 Univariate Analysis - Numerical

#### 1.5 a) monthly_rent statistics

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 5))

# Histogram
ax1 = sns.histplot(x=train_data['monthly_rent'], color='teal', ax=ax[0, 0])
ax1.set_xlabel('Monthly Rent')
ax1.set_ylabel('Count')

# Boxplot
ax2 = sns.boxplot(x=train_data['monthly_rent'], ax=ax[0, 1], color='teal')
ax2.set_xlabel('Monthly Rent')
ax2.set_ylabel('')

# Violin Plot
ax3 = sns.violinplot(x=train_data['monthly_rent'], ax=ax[1, 0], color='teal')
ax3.set_xlabel('Monthly Rent')
ax3.set_ylabel('Density')

# Probability Plot
ax4 = probplot(train_data['monthly_rent'], plot=ax[1, 1])
ax[1, 1].set_xlabel('Theoretical Quantiles')
ax[1, 1].set_ylabel('Ordered Values')
# Remove the title
ax[1, 1].set_title('')

plt.tight_layout()  # Adjust the layout
pylab.show()

#### 1.5 b) region v.s. monthly_rent

In [ ]:
sns.set_context('notebook', font_scale = 1.3)
plt.figure(figsize=(15, 4))
ax = sns.barplot(x=train_data['region'],
                 y=train_data['monthly_rent'],
                 palette=sns.color_palette("viridis", n_colors=len(train_data['region'].unique())),
                 ci=None)
plt.ylabel('Monthly Rent');

for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x() + 0.4, p.get_height() + 1), ha='center', va='bottom', color='Black')


#### 1.5 c) flat_type v.s. monthly_rent

In [ ]:
datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    # Replace "executive" with "6-room"
    dataset['flat_type'] = dataset['flat_type'].replace('executive', '5 room')
    # Extract the numeric part and convert it to integer
    dataset['room_num'] = dataset['flat_type'].str.extract('(\d)').astype(int)
    # Update the original datasets
    datasets[idx] = dataset
train_data, test_data = datasets

# Group by room_num to get average rent and average rent per room
grouped_data = train_data.groupby('room_num').agg({
    'monthly_rent': 'mean'
})
grouped_data['rent_per_room'] = grouped_data['monthly_rent'] / grouped_data.index

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 3))  # Adjust the figure size for better visualization

color = 'tab:blue'
ax1.set_xlabel('Room Number')
ax1.set_ylabel('Monthly Rent Mean', color=color)
ax1.plot(grouped_data.index, grouped_data['monthly_rent'], color=color, linestyle='-', marker='o')  # Added markers for clarity
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim([1500, 3000])  # Set y-axis limit for ax1 as specified

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Rent per Room Mean', color=color)
ax2.plot(grouped_data.index, grouped_data['rent_per_room'], color=color, linestyle='--', marker='x')  # Different linestyle and markers for distinction
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim([500, 1000])  # Set y-axis limit for ax2 as specified

# Add a title and grid for better visualization
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

fig.tight_layout()
plt.show()

#### 1.5 d) flat_model v.s. monthly_rent

In [ ]:
print(train_data['flat_model'].unique())

In [ ]:
model_mean_rent = train_data.groupby('flat_model')['monthly_rent'].mean().reset_index()
model_mean_rent = model_mean_rent.sort_values('monthly_rent')

sns.set_context('notebook', font_scale=1.3)
plt.figure(figsize=(13, 4))

# Create the barplot
ax = sns.barplot(x='flat_model',  # 'flat_model' column from the grouped DataFrame
                 y='monthly_rent',  # Mean 'monthly_rent' from the grouped DataFrame
                 data=model_mean_rent,  # The grouped DataFrame with mean rent
                 palette='viridis',
                 ci=None)

# Remove x-axis labels to replace them with annotations inside the bars
ax.set_xticklabels([])

ax.set_xlabel('Flat Model')
plt.ylabel('Mean Monthly Rent')

# Loop over each bar in the plot
for p in ax.patches:
    # Get the bar height and x-position
    height = int(p.get_height())
    x = p.get_x() + p.get_width() / 2

    # Annotate the height of the bar inside the bar
    ax.annotate(f'{height}', (x, height), ha='center', va='center', color='black', xytext=(0, 10), textcoords='offset points', fontweight='bold')

    # Annotate the flat_model name inside the bar
    flat_model_name = model_mean_rent.loc[p.get_x() + p.get_width() / 2, 'flat_model']  # Get the corresponding flat_model name
    ax.annotate(flat_model_name, (x, 0), ha='center', va='bottom', color='white', rotation=90, xytext=(0, 10), textcoords='offset points',fontweight='bold', fontsize=12.5)

# Show the plot
plt.show()

#### 1.5 e) house age v.s. monthly_rent

In [ ]:
import matplotlib.pyplot as plt

train_data = train_data.sort_values('lease_commence_date')

# Calculate real estate age without modifying the original train_data
age_rent_df = train_data[['lease_commence_date', 'monthly_rent']].copy()
age_rent_df['HDB_age'] = 2023 - age_rent_df['lease_commence_date']

# Group by real estate age and calculate mean rent
grouped_data = age_rent_df.groupby('HDB_age').agg({
    'monthly_rent': 'mean'
}).reset_index()

# Plotting
plt.figure(figsize=(10, 3))
plt.plot(grouped_data['HDB_age'], grouped_data['monthly_rent'], marker='o', linestyle='-')

# Highlight the point for lease_commence_date = 1966 in red
specific_year = 2023 - 1966
mean_rent_for_year = grouped_data[grouped_data['HDB_age'] == specific_year]['monthly_rent'].values[0]
plt.scatter(specific_year, mean_rent_for_year, color='orange', zorder=5)  # zorder=5 to draw the point on top of the line

plt.xlabel('Real Estate Age (Years)')
plt.ylabel('Mean Rent')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
train_data['lease_commence_date'].unique()

#### 1.5 f) floor_area_sqm v.s. monthly_rent

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))

sns.histplot(train_data['floor_area_sqm'], bins=30, kde=False, color='orange', ax=ax[0])  # Adjust position as desired
sns.scatterplot(x=train_data['floor_area_sqm'], y=train_data['monthly_rent'], ax=ax[1], color='green', alpha=0.5)  # Adjust position as desired

plt.tight_layout()
plt.show()

#### 1.5 g) monthly_rent per sqm

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))

# Left-hand side plot
sns.histplot(train_data['monthly_rent'] / train_data['floor_area_sqm'], bins=30, kde=False, color='orange', ax=ax[0])
ax[0].set_title('Distribution of Rent per sqm')  # Adding a title
ax[0].set_xlabel('Rent per sqm')  # Setting x-label
ax[0].set_ylabel('Frequency')  # Setting y-label

# Right-hand side plot
sns.scatterplot(x=train_data['floor_area_sqm'], y=train_data['monthly_rent'] / train_data['floor_area_sqm'], ax=ax[1], color='green', alpha=0.5)
ax[1].set_title('Rent per sqm vs. Floor Area')  # Adding a title
ax[1].set_xlabel('Floor Area (sqm)')  # Setting x-label
ax[1].set_ylabel('Rent per sqm')  # Setting y-label

plt.tight_layout()
plt.show()

#### 1.5 h) monthly_rent heat map

In [ ]:
# Create a base map
zoom_level=12
m = folium.Map(location=[train_data['latitude'].mean(), train_data['longitude'].mean()], zoom_start=zoom_level,
               zoomControl=False, max_zoom=zoom_level, min_zoom=zoom_level, width='100%', height='100%')

# Add the heatmap
heat_data = [[row['latitude'], row['longitude'], row['monthly_rent']] for index, row in train_data.iterrows()]
HeatMap(heat_data).add_to(m)
m

#### 1.5 i) monthly_rent clustering

In [ ]:
# # Create a base map
# zoom_level=12
# m = folium.Map(location=[train_data['latitude'].mean(), train_data['longitude'].mean()], zoom_start=zoom_level,
#                zoomControl=False, max_zoom=zoom_level, min_zoom=zoom_level, width='100%', height='100%')
#
# # Add the heatmap
# heat_data = [[row['latitude'], row['longitude'], row['monthly_rent']] for index, row in train_data.iterrows()]
# HeatMap(heat_data).add_to(m)
# m
#
# # Extract and standardize the longitude and latitude features
# coords = pd.concat([train_data[['longitude', 'latitude']], test_data[['longitude', 'latitude']]])
# coords_scaled = StandardScaler().fit_transform(coords)
#
# # K-Means clustering
# kmeans = KMeans(n_clusters=12, random_state=0)
# kmeans.fit(coords_scaled)
# train_data['kmeans_labels'] = kmeans.predict(train_data[['longitude', 'latitude']])
# test_data['kmeans_labels'] = kmeans.predict(test_data[['longitude', 'latitude']])
#
# # Plotting
# fig, ax = plt.pl(1, 2, figsize=(12, 5))
#
# # K-Means plot
# ax[1].scatter(train_data['longitude'], train_data['latitude'], c=train_data['kmeans_labels'], cmap='rainbow', s=50)
# ax[1].set_title('K-Means Clustering')
# ax[1].set_xlabel('Longitude')
# ax[1].set_ylabel('Latitude')
#
# plt.tight_layout()
# plt.show()

#### 1.5 j) monthly_rent mean and std

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'train_data' is your DataFrame and 'rent_approval_date' and 'monthly_rent' are columns in it.

# Copy the original train data to avoid modifying it
split_date = train_data.copy()

# Split the 'rent_approval_date' into separate year and month columns
split_date[['rent_year', 'rent_month']] = split_date['rent_approval_date'].str.split('-', expand=True).astype(int)

# Group by 'rent_year' and 'rent_month', then calculate mean and standard deviation for 'monthly_rent'
monthly_rent_stats = split_date.groupby(['rent_year', 'rent_month'])['monthly_rent'].agg(
    monthly_rent_mean='mean',
    monthly_rent_volatility='std'
).reset_index()

# Calculate the yearly average rent
yearly_rent_avg = split_date.groupby('rent_year')['monthly_rent'].mean().rename('monthly_rent_yearly_avg').reset_index()

# Merge the yearly average rent with the monthly stats
monthly_rent_stats = pd.merge(monthly_rent_stats, yearly_rent_avg, on='rent_year')

# Create a 'year_month' column for plotting purposes
monthly_rent_stats['year_month'] = monthly_rent_stats['rent_year'].astype(str) + '-' + monthly_rent_stats['rent_month'].astype(str)

# Create a plot
fig, ax = plt.subplots(figsize=(12, 4))

# Plot the mean of the monthly rent
ax.set_xlabel('Year-Month', fontsize=18)
ax.set_ylabel('Rent Price', fontsize=18)
ax.plot(monthly_rent_stats['year_month'], monthly_rent_stats['monthly_rent_mean'], color='tab:blue', label='Monthly Mean')
ax.plot(monthly_rent_stats['year_month'], monthly_rent_stats['monthly_rent_volatility'], color='tab:red', linestyle='--', label='Monthly Volatility')
ax.plot(monthly_rent_stats['year_month'], monthly_rent_stats['monthly_rent_yearly_avg'], color='tab:green', linestyle='-.', label='Yearly Average')
ax.tick_params(axis='y', labelsize=16)
ax.tick_params(axis='x', rotation=45, labelsize=16)
ax.legend()

# Title of the plot
# plt.title('Monthly Rent Mean, Yearly Average, and Volatility Over Time', fontsize=16)

# Show the plot
fig.tight_layout()  # To ensure the subplot fits into the figure area
plt.show()


In [ ]:
np.mean(train_data['monthly_rent'])

## 1.6 EDA - Aux Data

#### 1.6 a) coe
* To register a vehicle, you must first bid for a COE.Category refers to different vehicle types. Some background info: https://onemotoring.lta.gov.sg/content/onemotoring/home/buying/upfront-vehicle-costs/certificate-of-entitlement--coe-.html
1. category: Different categories might represent different types of vehicles, such as motorcycles, passenger cars, commercial vehicles, etc.
2. month: the month in which the COE bid took place.
3. bidding: the bidding round or session within the specified month.
4. price: the successful bid price for the COE in that category and bidding session. It represents the cost to obtain the COE.
5. quota: the number of COEs available for bidding in that category and session. It's the supply side of the equation.
6. bids: the number of bids received for the available COEs in that category and session. It's the demand side of the equation.

In [ ]:
# Combine 'year' and 'month' into a 'date' column and convert to datetime
coe_price_data['date'] = pd.to_datetime(coe_price_data['year'].astype(str) + ' ' + coe_price_data['month'], format='%Y %B')
coe_price_data.drop(columns=['year','month','bidding'], inplace=True)
coe_price_data.head()

In [ ]:
# Convert the "date" column to datetime format if it's not already
coe_price_data['date'] = pd.to_datetime(coe_price_data['date'])

# Extract year and month from the 'date' column
coe_price_data['year'] = coe_price_data['date'].dt.year
coe_price_data['month'] = coe_price_data['date'].dt.month

# Group by 'year', 'month', and 'category', then calculate the mean price
monthly_coe_stats = coe_price_data.groupby(['year', 'month', 'category'])['price'].mean().reset_index()

# Pivot the data to have categories as columns and fill NaNs with the previous value
category_pivot = monthly_coe_stats.pivot_table(index=['year', 'month'], columns='category', values='price').fillna(method='ffill')

# Group by 'rent_year' and 'rent_month', then calculate mean for 'monthly_rent'
monthly_rent_stats = split_date.groupby(['rent_year', 'rent_month'])['monthly_rent'].mean().reset_index()

# Combine the COE categories and rent price into one DataFrame for scaling
combined_stats = category_pivot.reset_index()
combined_stats['rent_price'] = monthly_rent_stats['monthly_rent']

# Standardize the data
scaler = StandardScaler()
combined_stats[['rent_price', 'a', 'b', 'c', 'e']] = scaler.fit_transform(combined_stats[['rent_price', 'a', 'b', 'c', 'e']])

# Create a 'year_month' column for plotting purposes
combined_stats['year_month'] = combined_stats['year'].astype(str) + '-' + combined_stats['month'].apply(lambda x: f'{x:02d}')

# Sort the DataFrame based on 'year_month' to ensure the plot is in chronological order
combined_stats.sort_values('year_month', inplace=True)

# Set the figure size for better readability
plt.figure(figsize=(12, 8))

# First subplot for category-wise COE price trends
plt.subplot(2, 1, 1)
colors = ['blue', 'green', 'red', 'cyan', 'magenta']
line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]
for i, category in enumerate(['a', 'b', 'c', 'e']):
    plt.plot(combined_stats['year_month'], combined_stats[category].rolling(window=3).mean(), label=f'Cat. {category.upper()}', color=colors[i], linestyle=line_styles[i], linewidth=2)
plt.plot(combined_stats['year_month'], combined_stats['rent_price'].rolling(window=3).mean(), color='yellow', label='Standardized Monthly Rent', linewidth=2)
plt.title('Standardized COE Price Trends by Category Year-Monthly', fontsize=18)
# plt.xlabel('Year-Month', fontsize=18)
plt.ylabel('Standardized Mean Value', fontsize=18)
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)

# Second subplot for mean COE price vs rent
plt.subplot(2, 1, 2)
plt.plot(combined_stats['year_month'], combined_stats[['a', 'b', 'c','e']].mean(axis=1).rolling(window=3).mean(), color='blue', label='Mean COE Price', linewidth=2)
plt.plot(combined_stats['year_month'], combined_stats['rent_price'].rolling(window=3).mean(), color='red', label='Standardized Monthly Rent', linewidth=2)
plt.title('Standardized Mean COE Price vs. Monthly Rent Year-Monthly', fontsize=18)
plt.xlabel('Year-Month', fontsize=18)
plt.ylabel('Standardized Mean Value', fontsize=18)
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Set the figure size for better readability
plt.figure(figsize=(10, 4))

# Define colors and line styles for the plot
colors = ['blue', 'green', 'red', 'cyan', 'magenta']
line_styles = ['--', '--', '--', '--', '--']  # Dashed lines for categories
solid_line_style = '-'  # Solid line for mean COE and rent

# Plot category-wise COE price trends
for i, category in enumerate(['a', 'b', 'c', 'e']):
    plt.plot(combined_stats['year_month'], combined_stats[category].rolling(window=3).mean(), label=f'Cat. {category.upper()}', color=colors[i], linestyle=line_styles[i], linewidth=2)

# Plot the mean COE price (solid line)
plt.plot(combined_stats['year_month'], combined_stats[['a', 'b', 'c', 'e']].mean(axis=1).rolling(window=3).mean(), color='black', label='Mean COE Price', linestyle=solid_line_style, linewidth=2)

# Plot the standardized monthly rent (solid line)
plt.plot(combined_stats['year_month'], combined_stats['rent_price'].rolling(window=3).mean(), color='yellow', label='Standardized Monthly Rent', linestyle=solid_line_style, linewidth=2)

# Title and labels
plt.title('Standardized COE Price Trends and Monthly Rent Year-Monthly', fontsize=18)
plt.xlabel('Year-Month', fontsize=18)
plt.ylabel('Standardized Mean Value', fontsize=18)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add legend and grid
plt.legend()
plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

#### 1.6 b) primary_school
1. singapore primary school choose: https://investingsg.com/keyinfo/schools-properties/. Key points:
* 什么样的房子算学区房？
    * 教育部说了 ”每一所小学都是好学校“。但是，民间排名一直都在。一般意义上是在名校一公里范围内，抽签概率好的学校可以扩大到两公里范围。如果做社区义工还要注意住址和小学是否在同一个GRC。
* 租房租多久，什么时候开始租？
    * 报名是按IC地址，所以没有明确开始租房日期，只要报名时IC地址改了就可以。报名成功后需住满30个月，这项规定同样适用于买的房子。

In [ ]:
primary_school_data.head()

#### 1.6 c) mall

In [ ]:
mall_data.head()

#### 1.6 d) mrt_planned
* https://www.propertyguru.com.sg/property-guides/mrt-effect-on-property-prices-39498
   * 0.3 KM: MRT-house
    * 1KM: MRT-near-house

In [ ]:
mrt_planned_data.head()

#### 1.6 e) mrt_exist

In [ ]:
mrt_exist_data.head()

#### 1.7 f) stock_prices

In [ ]:
stock_price_data['name'].value_counts()

In [ ]:
# remove illiquid stocks
mask = stock_price_data['name'].transform(lambda x: stock_price_data['name'].value_counts()[x] >= 100)
liquid_stock_price_data = stock_price_data[mask]

# compute monthly statistics
monthly_stats = get_economic_indicator_feature(liquid_stock_price_data, 'stock', 'adjusted_close')

# Create a new column 'year_month' for plotting purposes
monthly_stats['year_month'] = monthly_stats['year'].astype(str) + '-' + monthly_stats['month'].astype(str)

# Explicitly create a figure and set the size
fig, ax1 = plt.subplots(figsize=(12, 4))

# Plot the mean of the index on the primary y-axis
ax1.set_xlabel('Year-Month', fontsize=14)
ax1.set_ylabel('Index Mean', color='tab:blue', fontsize=18)
ax1.set_ylim(0,60)
line1 = ax1.plot(monthly_stats['year_month'], monthly_stats['index_price'], color='tab:blue', label='Index Mean')
ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.tick_params(axis='x', rotation=45, labelsize=14)

# Instantiate a second y-axis sharing the same x-axis
ax2 = ax1.twinx()
ax2.set_ylim(0,60)
ax2.set_ylabel('Index Volatility', color='tab:red', fontsize=18)
line2 = ax2.plot(monthly_stats['year_month'], monthly_stats['index_volatility'], color='tab:red', label='Index Volatility')
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=14)

# Title of the plot
# plt.title('Index Mean and Volatility Over Time')

# Combine the legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc=0, fontsize=18)

# Adjust the layout and show the plot
fig.tight_layout()
plt.show()

In [ ]:

# Plot the mean of the index on the primary y-axis
ax1.set_xlabel('Year-Month', fontsize=14)
ax1.set_ylabel('Index Mean', color='tab:blue', fontsize=18)
ax1.set_ylim(0,60)
line1 = ax1.plot(monthly_stats['year_month'], monthly_stats['index_price'], color='tab:blue', label='Index Mean')
ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.tick_params(axis='x', rotation=45, labelsize=14)

# Instantiate a second y-axis sharing the same x-axis
ax2 = ax1.twinx()
ax2.set_ylim(0,60)
ax2.set_ylabel('Index Volatility', color='tab:red', fontsize=18)
line2 = ax2.plot(monthly_stats['year_month'], monthly_stats['index_volatility'], color='tab:red', label='Index Volatility')
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=14)

# Title of the plot
# plt.title('Index Mean and Volatility Over Time')

# Combine the legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc=0, fontsize=18)

# Adjust the layout and show the plot
fig.tight_layout()
plt.show()

## 1.8 Observation - Main Data
1. monthly rent
* it's not heavily skewed, but there are definately outliers
* regions - the average rent across different regions are roughly the same
* flat_type - the average rent is positively correlated to the #room
* flat_model - those with extremely low frequency in flat_model usually have a high price
* lease_commence_date - graph indicates the rent is increasing yearly, and exhibit up and downs in each decade. this can be combined with the auxiliary data to analyze the influence of economics
* latitude&longitude - observe from a higher angle, there are definately groups

2. region
* the rent data across regions varies less than 1%, this feature is providing very limited infomation

3. town & planning_area
* these two features' value overlaps, and there are 3 extra values in planning_area. binary encoded with other geographical features?
4. flat_type
* 4/3/5 rooms are most popular, does that mean sufficient demand or supply? (may influence price)
5. furnished
* 100% furnished, can we drop this feature?
6. latitude & longitude
* there are actually naturally formed groups, how to group them properly?
7. subzone
* maybe can be used with the latitude * longitude?
8. block
* What is block_NO: https://www.quora.com/Why-did-Singapore-start-having-HDB-block-numbers-with-letters-e-g-172A-B-C
* To sum up: each number is a location, and the following letter is the age. However, we have location & lease date info, thus drop this
9. Rent/SQM
* the distribution shows a significant pattern compared to seperated; thus we should include this feature in FE
10. Grouping
* According to the heatmap, there are 12 natural groups.


## 1.9 Observation - Aux Data
1. coe_price_data
* figure out whether the COE is a lagging or not. More specifically, different types of COE may vary. Main focus should be on Category_C: Goods vehicle&bus, and Category_D: Motorcycle v.s. Category_A and Category_B. The former are used as productivity materials, while the latter is consumption.
2. primary_school_data
* for each school, find its KNN house, and statistics w.r.t rent: 1km_school_rent_mean, 1km_school_rent_median, 2km_school_rent_mode, ...;
* for each record, find all primary_schools within 1km, 2km range, and average out the statistics before-mentioned.
3. mall
* for each mall, find its KNN house, and the statistics: mall_average_rent, mall_median_rent, mall_75_rent, mall_25_rend;
* for each record, find the malls within 3km range, and average out the statistics before-mentioned.
4. mrt_planned
* for each record, find the #mrt_planned within 0.3KM and 1KM, with those beyond 3 years ones ignored.
5. mrt_exist
* for each record, find the #mrt_planned within 0.3KM and 1KM.
6. stock_price
* stock_price is a leading indicator of economics, while rent_price is lagging indicator. This relationship implies that stock_price can be used to predict the overall rent price.
* stock_price includes 59 stocks, thus we can form an index, and compute the index value on each trading day. However, those extremely illiquid stocks should be removed.
* Time-Series-Regression can have some weight in the final model.

# 2. Preprocessing
## 2.1 Eliminate Redundant Information

In [ ]:
# block: drop this feature, refer to EDA observations
# furnished: the value of this attribute for all data is "yes"
# elevation: Similar to attribute "furnished"
# region: drop this feature, refer to EDA observations
dropped_features = ['furnished', 'elevation', 'region', 'block']

datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    dataset.drop(columns=dropped_features, inplace=True)

train_data.head()

## 2.2 Remove Outlier

#### 2.2 a) lease_commence_date Outliers

In [ ]:
train_data = train_data[train_data['lease_commence_date'] >1966]

#### 2.2 b) monthly_rent Outliers

In [ ]:
# Calculate the IQR
Q1 = train_data['monthly_rent'].quantile(0.25)
Q3 = train_data['monthly_rent'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for the outliers
lower_bound = Q1 - 1.45 * IQR
upper_bound = Q3 + 2.5 * IQR

# Identify outliers
outliers = (train_data['monthly_rent'] < lower_bound) | (train_data['monthly_rent'] > upper_bound)
# Filter out the outliers
train_data = train_data[~outliers]
train_data.shape

In [ ]:
train_data['rent_per_sqm'] = train_data['monthly_rent'] / train_data['floor_area_sqm']
# Create a DataFrame for the regression
data_for_model = pd.DataFrame({
    'floor_area_sqm': train_data['floor_area_sqm'],
    'rent_per_sqm': train_data['rent_per_sqm']
})

# Fit the quantile regression model at the 95th percentile
quantile_model = smf.quantreg('rent_per_sqm ~ floor_area_sqm', data_for_model).fit(q=0.997)

# Predict the boundary values
train_data['boundary'] = quantile_model.predict(train_data[['floor_area_sqm']])

# Identify outliers
outliers = train_data['rent_per_sqm'] > train_data['boundary']

# Remove outliers from the original train_data DataFrame
train_data = train_data[~outliers].reset_index(drop=True)

train_data.drop(columns=['boundary','rent_per_sqm'], inplace=True)
train_data.shape

In [ ]:
# import statsmodels.formula.api as smf
# train_data_path = './dataset/train.csv'
# train_data = pd.read_csv(train_data_path)
#
# suns_color_primary_light = '#5D3FD3'  # Lighter Purple
# suns_color_secondary_light = '#FF8F66'  # Lighter Orange
# suns_color_accent_light = '#FFC266'  # Lighter Yellow
#
# # First plot data and quantile regression
# data = pd.DataFrame({
#     'floor_area_sqm': train_data['floor_area_sqm'],
#     'rent_per_sqm': train_data['monthly_rent'] / train_data['floor_area_sqm']
# })
#
# # Fit the quantile regression model at the 99.95th percentile
# quantile_model = smf.quantreg('rent_per_sqm ~ floor_area_sqm', data).fit(q=0.997)
#
# # Predict the values along the boundary
# data['prediction'] = quantile_model.predict(data[['floor_area_sqm']])
#
# # Identify outliers
# outliers_quantile = data['rent_per_sqm'] > data['prediction']
#
# # Second plot data and IQR
# Q1 = train_data['monthly_rent'].quantile(0.25)
# Q3 = train_data['monthly_rent'].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.45 * IQR
# upper_bound = Q3 + 2.5 * IQR
# outliers_iqr = (train_data['monthly_rent'] < lower_bound) | (train_data['monthly_rent'] > upper_bound)
#
# # Create a figure and a set of subplots
# fig, axs = plt.subplots(1, 2, figsize=(16, 6))
#
# # Plot 'floor_area_sqm' vs. 'rent_per_sqm' with outliers
# axs[1].scatter(data['floor_area_sqm'], data['rent_per_sqm'], alpha=0.5, color=suns_color_primary_light, label='Data')
# axs[1].plot(data['floor_area_sqm'], data['prediction'], color=suns_color_secondary_light, label='99.7th Quantile Regression Line')
# axs[1].scatter(data.loc[outliers_quantile, 'floor_area_sqm'], data.loc[outliers_quantile, 'rent_per_sqm'], alpha=0.9, color=suns_color_accent_light, label='Outliers')
# axs[1].set_xlabel('Floor Area (sqm)')
# axs[1].set_ylabel('Rent per Sqm')
# axs[1].set_title('Floor Area vs Rent per Sqm with Outliers')
# axs[1].legend()
#
# # Plot 'monthly_rent' with outliers
# axs[0].scatter(train_data.index, train_data['monthly_rent'], alpha=0.5, color=suns_color_primary_light, label='Data')
# axs[0].scatter(train_data.index[outliers_iqr], train_data['monthly_rent'][outliers_iqr], alpha=0.9, color=suns_color_accent_light, label='Outliers beyond [-1.45~2.5]*IQR')
# axs[0].set_xlabel('Index')
# axs[0].set_ylabel('Monthly Rent')
# axs[0].set_title('Monthly Rent Distribution with Outliers')
# axs[0].legend()
#
# # Adjust layout to prevent overlap
# plt.tight_layout()
#
# # Show the plot
# plt.show()

## 2.3 Reformat Feature Values

In [ ]:
train_data.head()

In [ ]:
cat_features = ['town', 'street_name', 'subzone', 'flat_model', 'planning_area']
datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    for feature in cat_features:
        dataset[feature] = dataset[feature].str.lower()

train_data.head()


# 3. Feature Engineering
1. Binning of numerical variables - https://www.kaggle.com/competitions/two-sigma-connect-rental-listing-inquiries/discussion/32116
2. create new features by encoding categorical variables using statistics (like mean, median, standard deviation, etc.) of continuous variables. https://www.kaggle.com/competitions/two-sigma-connect-rental-listing-inquiries/discussion/32123

### 3.1 Synergistically Combined Features

#### 3.1 a) Date Related Features
1. lease_commence_date -> HDB_age
2. rent_approval_date -> rent_year, rent_month

In [ ]:
datasets = [train_data, test_data]

for idx, dataset in enumerate(datasets):
    # lease_commence_date -> HDB_age
    dataset['HDB_age'] = 2023 - dataset['lease_commence_date']

    # rent_approval_date -> rent_year, rent_month
    split_date = dataset['rent_approval_date'].str.split('-')
    dataset['rent_year'] = split_date.str[0].astype('int32')
    dataset['rent_month'] = split_date.str[1].astype('int32')

    dataset.drop(columns=['rent_approval_date'], inplace=True)
    dataset.drop(columns=['lease_commence_date'], inplace=True)

    # Update the original datasets
    datasets[idx] = dataset

train_data, test_data = datasets

train_data[:5]

#### 3.1 b) Encode the Urban Planning Areas
* Planning Area (e.g., Bedok, Tampines, Jurong East)
    * Subzone (e.g., Bedok North, Tampines East, Jurong Gateway)
        * Town (e.g., Tampines Town, which may overlap with the Tampines Planning Area)
                * Street (e.g., Tampines Street 11, which is a road within Tampines Town)

In [ ]:
# encode the urban planning
urban_planning = ['planning_area', 'subzone', 'town']
train_data, test_data = train_data, test_data = encode_hierarchical(train_data, test_data, urban_planning)

print(train_data[['planning_area', 'subzone', 'town', 'city_encoder']].head())

#### 3.1 c) Room Num

In [ ]:
datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    # Replace "executive" with "6-room"
    dataset['flat_type'] = dataset['flat_type'].replace('executive', '6 room')
    # Extract the numeric part and convert it to integer
    dataset['room_num'] = dataset['flat_type'].str.extract('(\d)').astype(int)
    # Update the original datasets
    dataset.drop(columns=['flat_type'], inplace=True)
    datasets[idx] = dataset
train_data, test_data = datasets

train_data.head()

### 3.2 Neighboring Facilities Features

#### 3.2 a) "Eigenvector Centrality" Statistics
* for each record of each aux-data: find its KNN houses and calculate statistics
* for each house, find its KNN aux-record, and weighted-average their statistics

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def weighted_avg(values, weights):
    """Compute the weighted average of values."""
    return np.average(values, weights=weights)

def calculate_distance_to_mrt(aux_data, mrt_data, k=3):
    """Calculate the mean distance of each auxiliary data point to the k nearest MRTs."""
    aux_data_rad = np.radians(aux_data[['latitude', 'longitude']])
    mrt_data_rad = np.radians(mrt_data[['latitude', 'longitude']])

    knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree', metric='haversine')
    knn.fit(mrt_data_rad)

    distances, _ = knn.kneighbors(aux_data_rad)
    mean_distances = distances.mean(axis=1)  # Calculate the mean distance to the k nearest MRTs

    return mean_distances

def get_knn_values(base_data, aux_data, k, mrt_data):
    # Convert latitude and longitude to radians for haversine metric
    base_data_rad = np.radians(base_data[['latitude', 'longitude']])

    # Calculate mean distance to the k nearest MRTs for auxiliary data
    aux_data['value'] = calculate_distance_to_mrt(aux_data, mrt_data, k)

    # Initialize KNN for base data
    knn_base = NearestNeighbors(n_neighbors=k, algorithm='ball_tree', metric='haversine')
    knn_base.fit(base_data_rad)

    # Compute the weighted mean 'value' of KNN schools and malls for each block
    weighted_values = []  # This will store the weighted mean values for each row in base_data
    for index, row in base_data.iterrows():
        # Find KNN schools/malls for the block
        distances, indices = knn_base.kneighbors([np.radians([row['latitude'], row['longitude']])])
        distances = distances.flatten() + 1e-10  # Avoid division by zero
        indices = indices.flatten()

        # Ensure indices are within the bounds of aux_data
        valid_indices = indices[indices < len(aux_data)]
        if len(valid_indices) > 0:
            knn_values = aux_data.iloc[valid_indices]['value']
            weighted_mean_value = weighted_avg(knn_values, 1 / distances[:len(valid_indices)])
        else:
            weighted_mean_value = 0
        weighted_values.append(weighted_mean_value)

    return pd.Series(weighted_values)  # Return a Series of weighted mean values

datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    dataset['school_value'] = get_knn_values(dataset, primary_school_data, k=5, mrt_data=mrt_exist_data)
    dataset['mall_value'] = get_knn_values(dataset, mall_data, k=5, mrt_data=mrt_exist_data)
    datasets[idx] = dataset

train_data, test_data = datasets
train_data.head()

#### 3.2 b) Lifecycle Facility Counts
* compute the trans-cycles according to the gov plan
    * MRT: 0.3 KM  MRT-house, 1KM: MRT-near-house
    * Primary School: not explicit data, assume the 15min driving cycle of a school bus, 5KM
    * Mall: 15min driving cycle, approximately 8KM

In [ ]:
from sklearn.neighbors import NearestNeighbors

def count_nearby_records(base_data, aux_data, radius):
    """Count the number of nearby records within a specified radius."""

    # Convert latitude and longitude to radians for haversine metric
    base_data_rad = np.radians(base_data[['latitude', 'longitude']])
    aux_data_rad = np.radians(aux_data[['latitude', 'longitude']])

    # Initialize KNN
    knn = NearestNeighbors(algorithm='ball_tree', metric='haversine')
    knn.fit(aux_data_rad)

    # Find records within the specified radius
    indices = knn.radius_neighbors(base_data_rad, radius=radius/6371.0, return_distance=False)  # radius/6371.0 to convert km to radians

    # Count the number of records within the radius for each main record
    counts = [len(ind) for ind in indices]

    return counts

# Define radii for each auxiliary dataset
radii = {
    "school": 5.0,  # 5 km
    "mrt_adjacent": 0.3,  # 0.3 km
    "mrt_nearby": 1.0,  # 1 km
    "mall": 8.0  # 8 km
}

# Process datasets
datasets = [train_data, test_data]
aux_datasets = {"school": primary_school_data,
                "mrt_plan": mrt_exist_data,
                "mrt_exist": mrt_exist_data,
                "mall": mall_data}

for idx, dataset in enumerate(datasets):
    dataset[f'school_count'] = count_nearby_records(dataset, aux_datasets["school"], radii["school"])
    dataset[f'mall_count'] = count_nearby_records(dataset, aux_datasets["mall"], radii["mall"])

    # For MRT stations
    for mrt_key in ["mrt_plan", "mrt_exist"]:
        for radius_key in ["mrt_adjacent", "mrt_nearby"]:
            col_name = f'{mrt_key}_{radii[radius_key]}_count'
            dataset[col_name] = count_nearby_records(dataset, aux_datasets[mrt_key], radii[radius_key])

    # Update the original datasets
    datasets[idx] = dataset

train_data, test_data = datasets
train_data.head()

#### 3.2 c) Min Distance to Facilities

In [ ]:
def haversine_distance_rad(lat1, lon1, lat2, lon2):
    """
    Calculate the haversine distance between two sets of latitude and longitude coordinates in radians.
    """
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

datasets = [train_data, test_data]
aux_datasets = {"school": primary_school_data,
                "mrt_plan": mrt_exist_data,
                "mrt_exist": mrt_exist_data,
                "mall": mall_data}

# For each auxiliary dataset, compute the minimum distance to each record in train_data and test_data
for aux_name, aux_data in aux_datasets.items():
    # Convert latitude and longitude to radians for haversine metric
    aux_data_rad = np.radians(aux_data[['latitude', 'longitude']])

    for dataset in datasets:
        distances = []
        base_data_rad = np.radians(dataset[['latitude', 'longitude']])

        for index, row in base_data_rad.iterrows():
            # Calculate distance from current record to all records in the auxiliary dataset
            dist = haversine_distance_rad(row['latitude'], row['longitude'], aux_data_rad['latitude'].values, aux_data_rad['longitude'].values)
            # Append the minimum distance to the distances list
            distances.append(dist.min())

        # Add the minimum distances as a new feature to the dataset
        dataset[f'min_distance_to_{aux_name}'] = distances

train_data.head()

### 3.3 Economic Indicator Features

#### 3.3 a) Stock index performance
1. remove stocks with limited transcations
2. compute the monthly index price and volatility based on adjusted_close_price
3. concat to train/test where rent_month = month, rent_year = year

In [ ]:
datasets = [train_data, test_data]

for idx, dataset in enumerate(datasets):
    # Merge train_data with monthly_stats
    dataset = dataset.merge(
        monthly_stats,
        left_on=['rent_year', 'rent_month'],
        right_on=['year', 'month'],
        how='left'
    )
    # Drop the 'year' and 'month' columns from merged_data as they are redundant
    dataset.drop(columns=['year', 'month','year_month'], inplace=True)
    # Update the original datasets
    datasets[idx] = dataset

train_data, test_data = datasets

train_data.head()

#### 3.3 b) COE trend

In [ ]:
# First, calculate the mean COE price across categories for each year and month
mean_coe_by_month = monthly_coe_stats.groupby(['year', 'month'])['price'].mean().reset_index()

datasets = [train_data, test_data]
for idx, dataset in enumerate(datasets):
    # Merge train_data with monthly_stats
    dataset = dataset.merge(
        mean_coe_by_month,
        left_on=['rent_year', 'rent_month'],
        right_on=['year', 'month'],
        how='left'
    )
    # Rename the 'price' column to 'mean_coe_price' to avoid confusion
    dataset.rename(columns={'price': 'mean_coe_price'}, inplace=True)
    # Drop the 'year' and 'month' columns from merged_data as they are redundant
    dataset.drop(columns=['year', 'month'], inplace=True)
    # Update the original datasets
    datasets[idx] = dataset

train_data, test_data = datasets
train_data.head()

### 3.4 Statistically Encoded Features
* group the dataframe for each categorical feature
* compute each group's statistics and merge to the dataframe

#### 3.4 a) Encoding w.r.t. 'floor_area_sqm' Statistics

In [ ]:
datasets = [train_data, test_data]
cat_features = ['flat_model', 'street_name']

encoder = GroupedStatsEncoder(target_column='floor_area_sqm', group_columns=cat_features)
encoder.fit(train_data)
for idx, dataset in enumerate(datasets):
    datasets[idx] = encoder.transform(dataset)

train_data, test_data = datasets
train_data.head()

#### 3.4 b) Encoding w.r.t. 'HDB_age' Statistics

In [ ]:
datasets = [train_data, test_data]

encoder = GroupedStatsEncoder(target_column='HDB_age', group_columns=cat_features)
encoder.fit(train_data)
for idx, dataset in enumerate(datasets):
    datasets[idx] = encoder.transform(dataset)

train_data, test_data = datasets
train_data.head()

### 3.5 Likelihood Encoded Features

#### 3.5 a) P(rent/sqm | Categorical Feature)

In [ ]:
datasets = [train_data, test_data]

# compute unit price
train_data['rent_per_sqm'] = train_data['monthly_rent'] / train_data['floor_area_sqm']
test_data['rent_per_sqm'] = test_data['monthly_rent'] / test_data['floor_area_sqm']

# Compute and merge statistics for each categorical column
for cat_feature in urban_planning:
    encoder = LikelihoodEncoder(cat_feature, target_feature='rent_per_sqm', alpha=1, noise_std=0.1)
    encoder.fit(train_data)
    for idx, dataset in enumerate(datasets):
        dataset = encoder.transform(dataset)

for cat_feature in urban_planning:
    encoder = LikelihoodEncoder(cat_feature, target_feature='monthly_rent', alpha=1, noise_std=0.1)
    encoder.fit(train_data)
    for idx, dataset in enumerate(datasets):
        dataset = encoder.transform(dataset)

for idx, dataset in enumerate(datasets):
    dataset.drop(columns=cat_features, inplace=True)
    dataset.drop(columns=['planning_area', 'subzone', 'town'], inplace=True)
    dataset.drop(columns='rent_per_sqm', inplace=True)

train_data, test_data = datasets
train_data.head()

## 3.6 Clustering as a Feature

#### 3.6 a) K-Means

In [ ]:
# # Fit the K-Means model on the standardized train_data
# coords_train = train_data[['longitude', 'latitude']]
# scaler = StandardScaler()
# coords_train_scaled = scaler.fit_transform(coords_train)
#
# kmeans = KMeans(n_clusters=12, init='k-means++', random_state=0, n_init=1)
# kmeans.fit(coords_train_scaled)
#
# # Assign cluster labels to train_data using the trained K-Means model
# train_data['cluster'] = kmeans.predict(coords_train_scaled)
#
# # Extract and standardize the 'longitude' and 'latitude' features from test_data
# coords_test = test_data[['longitude', 'latitude']]
# coords_test_scaled = scaler.transform(coords_test)  # Use the same scaler object that was fit on train_data
#
# # Assign cluster labels to test_data using the trained K-Means model
# test_data['cluster'] = kmeans.predict(coords_test_scaled)
#
# train_data, test_data = datasets
# train_data.head()

## 3.7 Magic Feature from Freature Importance

In [ ]:
train_data.head()

In [ ]:
feature_pairs = determine_feature_crosses(train_data, 'monthly_rent', 3)
# Now create the new crossed features in the DataFrame
train_data = create_feature_crosses(train_data, feature_pairs)
test_data = create_feature_crosses(test_data, feature_pairs)
train_data.head()

## 3.8 Feature Selection

In [ ]:
nan_counts = test_data.isna().sum()
features_with_nan = nan_counts[nan_counts > 0]
print(train_data.shape, test_data.shape, features_with_nan)

train_data.drop(columns=['latitude', 'longitude'], inplace=True)
test_data.drop(columns=['latitude', 'longitude'], inplace=True)

#### 3.7 a) PCA

In [ ]:
#### 2.4 a) PCA
train_data_without_target = train_data.drop(columns=['monthly_rent'])

# 1. Standardize the data
scaler = StandardScaler()
train_data_standardized = scaler.fit_transform(train_data_without_target)

# 2. Perform PCA
pca = PCA()
pca_result = pca.fit(train_data_standardized)

# 3. Calculate the explained variance ratio for each feature
explained_variance_ratio = pca_result.explained_variance_ratio_
# Calculate the cumulative sum of the explained variance ratio
cumulative_variance = np.cumsum(explained_variance_ratio)
# Find the number of components required to explain at least 90% of the variance
num_components = np.where(cumulative_variance >= 0.90)[0][0] + 1

# Create a DataFrame to display feature contributions
feature_contributions = pd.DataFrame({
    'Feature': train_data_without_target.columns,
    'Contribution to Variance': explained_variance_ratio
})
print(feature_contributions)

# most_contribute_feature
most_contribute_feature = np.array(feature_contributions['Feature'])

# Plotting the explained variance ratio
feature_names = most_contribute_feature

# Plotting the explained variance ratio with feature names
plt.figure(figsize=(11, 5))  # Increase figure size for clarity
colors = plt.cm.viridis(np.linspace(0, 1, len(explained_variance_ratio)))  # Generate a color map
bars = plt.bar(range(len(explained_variance_ratio)), explained_variance_ratio, alpha=0.7, align='center', color=colors)
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')

# Add lines and text for feature names
for idx, (bar, feature) in enumerate(zip(bars, feature_names)):
    yval = bar.get_height()
    line_height = 0.01
    text_position = yval + line_height + 0.01  # Adjust text position above the line
    line_color = 'white' if idx == 0 else 'grey'  # Set line color to white for the first index
    text_color = 'white' if idx == 0 else 'black'  # Set text color to white for the first index

    if idx < 2 and yval > 0.02:  # Assuming the second bar is also tall enough
        text_color = 'white'
    else:
        text_color = 'black'

    # Special adjustment for the first bar if it's too high
    if idx in [0, 1] and yval + line_height + 0.05 > plt.ylim()[1]:
        line_height = -0.04  # Draw line downwards
        text_position = yval + line_height - 0.04 # Place text below the line

    plt.plot([idx, idx], [yval, yval + line_height], color=line_color, lw=0.4)  # Draw a line from the bar
    plt.text(idx, text_position, feature, ha='center', rotation=90, fontsize=14, color=text_color, fontweight='bold')  # Place the text at the end of the line

plt.tight_layout()  # Adjust the layout to make room for the rotated labels
plt.show()

#### 3.7 b) Top-K Selection

In [ ]:
def get_top_k_features(dataset, k, standard=False):
    """
    Returns a DataFrame with the top-k most contributing features from train_data
    and the target feature 'monthly_rent'.

    Parameters:
    - k (int): Number of top features to select.
    - standard (Boolean): Whether to standardized or not

    Returns:
    - DataFrame: A DataFrame with the top-k features and the target feature.
    """

    # Top-K
    top_k_features = list(most_contribute_feature[:k])
    # Target
    top_k_features.append('monthly_rent')

    if standard:
        # Separate the target feature
        target_feature = dataset['monthly_rent'].copy()

        # Remove the target feature for standardization
        train_data_without_target = dataset.drop(columns=['monthly_rent'])

        train_data_standardized = StandardScaler().fit_transform(train_data_without_target)

        # Convert the array back to a DataFrame
        train_data_standardized_df = pd.DataFrame(train_data_standardized,
                                                  columns=train_data_without_target.columns)

        # Reset indices and add the target feature back to the standardized DataFrame
        train_data_standardized_df = train_data_standardized_df.reset_index(drop=True)
        target_feature = target_feature.reset_index(drop=True)
        train_data_standardized_df['monthly_rent'] = target_feature

    return dataset[top_k_features].copy()

# test_na_set = get_top_k_features(train_data, 20, True)
# nan_counts = test_na_set.isna().sum()
# features_with_nan = nan_counts[nan_counts > 0]
# print(features_with_nan)

#### 3.7 c) Variance contribution perccentage selection

In [ ]:
def get_var_percentage_features(dataset, percentage, standard=False):
    # Sum up contribution to variance
    total_variance = np.sum(feature_contributions['Contribution to Variance'])

    # Calculate the cumulative contribution to variance in percentage
    feature_contributions['Cumulative Contribution'] = feature_contributions['Contribution to Variance'].cumsum() / total_variance * 100

    # Find the number of features where the cumulative contribution is just over 90%
    k = feature_contributions[feature_contributions['Cumulative Contribution'] <= percentage].shape[0] + 1

    return get_top_k_features(dataset, k, standard)

# test_df = get_var_percentage_features(train_data, 95)
# test_df.head()

# 3. Save and load data

In [ ]:
# Save data into file
train_data.to_csv('preprocessed/train_data.csv', index=0)
test_data.to_csv('preprocessed/test_data.csv', index=0)
np.save('preprocessed/most_contribute_feature.npy', most_contribute_feature, allow_pickle=True)
feature_contributions.to_csv('preprocessed/feature_contributions.csv', index=0)

In [ ]:
# Load data from file
train_data = pd.read_csv('preprocessed/train_data.csv')
test_data = pd.read_csv('preprocessed/test_data.csv')
most_contribute_feature = np.load('preprocessed/most_contribute_feature.npy', allow_pickle=True)
feature_contributions = pd.read_csv('preprocessed/feature_contributions.csv')

# 4. Methods - Single Regressors

After EDA and data preprocessing, we can now apply data mining methods for prediction. In this section, we choose different algorithms to generate regression models, including Multiple Linear Regression, Random Forest, XGBoost and LightGBM. The general proposal is to split the preprocessed dataset into train/test data, train and validate with train data, and evaluate the optimized model performance with test data. Finally, compare the models to get a overall picture of how well these regression models perform.

Different modelling methods may involve different data processing and feature engineering techniques. Therefore, instead of using the train_data/test_data directly, we create separate copies of the dataset for each method to generate the corresponding training/testing data for that particular method.

### 4.0 General Evaluate Functions

In [ ]:
def plot_predict_vs_actual(y_actual, y_predict):
    plt.scatter(y_actual, y_predict)
    plt.xlabel("Actual Price")
    plt.ylabel("Predicted Price")
    plt.title("Actual vs Predicted")
    plt.show()

## 4.1 Multiple Linear Regression

### Training

In [ ]:
# Get var percentage 90 features
data_mlr = get_var_percentage_features(train_data, 90, True)

# Split data into features and target
X = data_mlr.drop(['monthly_rent'], axis=1)
y = data_mlr['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Linear Regression
mlr = LinearRegression()

# Fit the model
mlr = mlr.fit(X_train, y_train)

### Evaluation

#### Evaluate the model on training dataset

In [ ]:
y_pred_train = mlr.predict(X_train)

# Model evaluation metrics
print('R^2:',metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

**Assumptions on MLR Models**

There are some main assumptions on Multiple Regression Models, which are listed below:

* Linearity
* No Multi-Collinearity
* Homoskedasticity
* Independence of independent variable
* Normality
* Independence of errors

We should conduct examinations on whether the assumptions above are satisfied in this context. Basic methods include using diagnostic plots for the relevant analysis. For example, we can use residual plots to check if homoskedasticity is violated or the errors' independency. Also, scatter plots of dependent versus independent variables can help us find non-linear relationships and so on.

* **Linearity**

Linearity in MLR means that the relationship between the independent variables (features) and the dependent variable (target) is linear.

In [ ]:
fig, axs = plt.subplots(ncols=5, nrows=4, figsize=(25, 16))

axs = axs.flatten()
for i, k in enumerate(X_train.columns):
    sns.regplot(y=y_train, x=X_train[k], ax=axs[i], scatter_kws={"color": "teal"}, line_kws={"color": "red"})
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

We can observe from the plots that the relationship between the independent variables (features) and the dependent variable (target) is linear.

* **No Multi-Collinearity**

Multicollinearity occurs when two or more independent variables in the MLR model are highly correlated, making it difficult to separate their individual effects on the dependent variable. It can impact the reliability of your regression model's coefficients and predictions.

We use the Correlation Matrix to have a look at the multi-collinearity across all variables first. Look for high correlation coefficients (typically greater than 0.7) between pairs of variables. High correlations suggest multicollinearity.

In [ ]:
# Display Correlation Matrix
corrMatrix = X_train.corr()
sns.heatmap(corrMatrix, annot=False)
plt.show()

`flat_type` and `floor_area_sqm` has a correlation coefficient of 0.95, and `town` and `planning_area` has a correlation coefficient of 0.97. This may suggest multicollinearity.

Note that `flat_type`, `town` and `planning_area` are  encoded and is not numeric values by nature, so we may consider other encoding methods for avoiding multi-collinearity across the variables.

Then we use another metrics Variance Inflation Factor (VIF) to measure how much the variance of the estimated regression coefficients is increased due to multi-collinearity. Calculate the VIF for each independent variable. High VIF values (typically greater than 10) indicate multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame({
    'variables': X_train.columns,
    'VIF': [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
}).sort_values(by='VIF', ascending=True)

print(vif.head(10))

As mentioned above, high VIF values (typically greater than 10) indicate multicollinearity. The form shows features "flat_type", "floor_area_sqm", "HDB_age", "rent_year", "school_count" and "mall_count" may have issues of multicollinearity, among which the VIF value of "rent_year" is especially high, suggesting the potential inner problem.

* **Homoskedasticity**

Homoscedasticity means that the variance of the residuals (the differences between the observed values and the predicted values) is constant across all levels of the independent variables. If homoscedasticity is violated, it can lead to problems such as biased coefficient estimates and incorrect inference.

In [ ]:
plt.figure(figsize=(40, 32))
for i, k in enumerate(X_train.columns):
    plt.subplot(5, 4, i + 1)
    plt.scatter(X_train[k], y_train - y_pred_train)
    plt.xlabel(k)
    plt.ylabel('Residuals')

We can observe that there is a random scattering of points with no discernible pattern. As in a homoskedastic dataset, the points should be evenly scattered around the horizontal line at 0 (the residuals have constant variance), we can come to the conclusion that the Homoskedasticity is not violated.

Also, we do not observe a funnel-shaped, fan-shaped pattern, or any other systematic change in the spread of residuals as the fitted values change, which suggests heteroskedasticity (the variance of residuals is not constant).

* **Independence of independent variables**

MLR assumes that independent variables are not perfectly correlated with each other (i.e., they are independent). Violations of this assumption can lead to unstable coefficient estimates and difficulties in interpreting their individual effects. We perform check of the independence, and some typical ones are shown below.

In [ ]:
# Independence of X variables-'floor_area_sqm'
plt.scatter(X_train['floor_area_sqm'], y_train - y_pred_train)
plt.title("Floor Area Size vs Residuals")
plt.xlabel("floor_area_sqm")
plt.ylabel("Residuals")
plt.show()

In [ ]:
# Independence of X variables-'latitude'
# plt.scatter(X_train['latitude'], y_train - y_pred_train)
# plt.title("Latitude vs Residuals")
# plt.xlabel("latitude")
# plt.ylabel("Residuals")
# plt.show()

In [ ]:
# Independence of X variables-'longitude'
# plt.scatter(X_train['longitude'], y_train - y_pred_train)
# plt.title("Longitude vs Residuals")
# plt.xlabel("latitude")
# plt.ylabel("Residuals")
# plt.show()

The plots above all show that residuals do appear randomly and symmetrically distributed around zero under all conditions, which prove the independence.

* **Normality**

MLR assumes that the residuals (the differences between the observed and predicted values) are normally distributed. Deviations from normality can impact the validity of statistical inference, such as hypothesis tests and confidence intervals.

In [ ]:
# Normality of residuals
sns.distplot(y_train - y_pred_train)
plt.title("Normality of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

We can see from the plot above that the residuals are normally distributed, which means the normality assumption is satiefied.

* **Independence of errors**

Independence of errors states that the residuals (the differences between the observed values and the predicted values) should be independent of each other. Violations of this assumption can lead to incorrect parameter estimates, unreliable hypothesis tests, and inaccurate predictions.

Here we use *Durbin-Watson Test* for checking independence of errors. This statistical test checks for the presence of autocorrelation in the residuals. A Durbin-Watson statistic value close to 2 indicates no autocorrelation.

In [ ]:
from statsmodels.stats.stattools import durbin_watson
durbin_watson_statistic = durbin_watson(y_train - y_pred_train)
print(f'Durbin-Watson Statistic: {durbin_watson_statistic}')

The Durbin-Watson statistic value is close to 2, which means there is no autocorrelation (i.e. the assmuption is satisfied).

#### Evaluate the model on test dataset

After the evalution on the training dataset, as well as the checks for MLR assmuptions and corresponding model refinement, we can use the model as the final  MLR model and evaluate its performance on the test dataset.

In [ ]:
y_pred_test = mlr.predict(X_test)

# Model evaluation metrics
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

### Analysis

By looking at the training and testing result of the multiple linear regression model, we could find that the R^2 and RMSE is not very ideal. This might due to we use a very intuitive selection of features, and the simple structure of linear regression model could not handle the complexity of such multiple dimensions. Also we have noticed that there are inner problems of the multiple linear regression model such as the multi-collinearity, which suggests a single MLR model is not an potimal choice in the current context. But still we could base on this model to provide insights on other models, for example the selection of the input features.

## 4.2 Support Vector Regression

### Training

In [ ]:
# Get var percentage 90 features with standardization
data_svr = get_var_percentage_features(train_data, 90, True)

# Split data into features and target
X = data_svr.drop(['monthly_rent'], axis=1)
y = data_svr['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Support Vector Regression with appropriate parameters
# svr = SVR(kernel='linear', C=1.0, epsilon=0.2)
svr = SVR(max_iter=10000)

# Fit the model
svr = svr.fit(X_train, y_train)

### Evaluating

In [ ]:
# Predict on training set
y_pred_train = svr.predict(X_train)

# Display training set evaluation metrics
print('R^2:',metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

# Predict on testing set
y_pred_test = svr.predict(X_test)

# Display testing set evaluation metrics
print()
print('Testing Metrics:')
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

## 4.3 Random Forest

### Training

In [ ]:
# Get var percentage 90 features
data_rf = get_var_percentage_features(train_data, 90)

# Split data into features and target
X = data_rf.drop(['monthly_rent'], axis = 1)
y = data_rf['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Define XGBoost hyperparameters
# We use GridSearchCV as a technique to systematically search through different combinations of hyperparameters
rf_param_grid = {
    'max_depth': [80, 90, 100], # Maximum number of levels in each decision tree
    'max_features': [2, 3], # Maximum number of features considered for splitting a node
    'min_samples_leaf': [1, 3, 4, 5], # Minimum number of data points allowed in a leaf node
    'n_estimators': [100, 300, 600], # Number of trees in the forest
}

# Initialize RandomForest regressor and GridSearchCV
rf_reg = RandomForestRegressor(random_state = 42)
rf_grid = GridSearchCV(estimator = rf_reg, param_grid = rf_param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
rf_grid = rf_grid.fit(X_train, y_train)

# Display best hyperparameters
print()
print('Best hyperparameters:')
print(rf_grid.best_params_)

# Get the best RF estimator based on best parameters generated from above
rf_best = rf_grid.best_estimator_

### Evaluating

In [ ]:
# Predict on training set
y_pred_train = rf_best.predict(X_train)

# Display training set evaluation metrics
print('Training Metrics:')
print('R^2:',metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

# Predict on testing set
y_pred_test = rf_best.predict(X_test)

# Display testing set evaluation metrics
print()
print('Testing Metrics:')
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

# Display feature importances
rf_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance Score': rf_best.feature_importances_
}).sort_values(by='Importance Score', ascending=False)
print()
print('Top 10 important features:')
print(rf_features.head(10))

### Predict on test data

In [ ]:
# Get var percentage 90 features
test_data_rf = get_var_percentage_features(test_data, 90)
test_data_rf = test_data_rf.drop(['monthly_rent'], axis = 1)

# Predict on training set
pred_test_rf = rf_best.predict(test_data_rf)

# Save to file
pred_test_rf_df = pd.DataFrame()
pred_test_rf_df['id'] = [i for i in range(len(test_data))]
pred_test_rf_df['Predicted'] = pred_test_rf
pred_test_rf_df.to_csv('predict_result_rf.csv', index=0)

## 4.4 Gradient Boost Regressor

### Training

In [ ]:
# Get var percentage 90 features
data_gb = get_var_percentage_features(train_data, 90)

# Split data into features and target
X = data_gb.drop(['monthly_rent'], axis=1)
y = data_gb['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define GBoost hyperparameters
gb_param_grid = {
    'loss': ['quantile', 'absolute_error', 'squared_error', 'huber'],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 8, 10],
    'min_samples_leaf': [100, 150, 250],
    'max_features': [0.3, 0.1]
}

# Initialize GradientBoostingRegressor
gb_reg = GradientBoostingRegressor()
gb_grid = GridSearchCV(estimator=gb_reg, param_grid=gb_param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=2)

# Fit the model
gb_grid = gb_grid.fit(X_train, y_train)

# Display best hyperparameters
print()
print('Best hyperparameters:')
print(gb_grid.best_params_)

# Get best estimator
gb_best = gb_grid.best_estimator_

### Evaluating

In [ ]:
# Predict on training set
y_pred_train = gb_best.predict(X_train)

# Display training set evaluation metrics
print('Training Metrics:')
print('R^2:', metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:', 1 - (1 - metrics.r2_score(y_train, y_pred_train)) * (len(y_train) - 1) / (len(y_train) - X_train.shape[1] - 1))
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

# Predict on testing set
y_pred_test = gb_best.predict(X_test)

# Display testing set evaluation metrics
print()
print('Testing Metrics:')
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

# Display feature importances
gb_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance Score': gb_best.feature_importances_
}).sort_values(by='Importance Score', ascending=False)
print()
print('Top 10 important features:')
print(gb_features.head(10))

## 4.5 XGBoost Regressor

### Training

In [ ]:
# Get var percentage 90 features
data_xgb = get_var_percentage_features(train_data, 90)

# Split data into features and target
X = data_xgb.drop(['monthly_rent'], axis=1)
y = data_xgb['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost hyperparameters
xgb_param_grid = {
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [6, 8, 9, 10],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3],
    "colsample_bytree": [0.3, 0.4, 0.6, 0.8]
}

# Initialize XGBoost regressor and GridSearchCV
xgb_reg = xgb.XGBRegressor(seed=42, objective='reg:squarederror')
xgb_grid = GridSearchCV(estimator=xgb_reg, param_grid=xgb_param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
xgb_grid = xgb_grid.fit(X_train, y_train)

# Display best hyperparameters
print()
print('Best hyperparameters:')
print(xgb_grid.best_params_)

# Get best estimator
xgb_best = xgb_grid.best_estimator_

### Evaluating

In [ ]:
# Predict on training set
y_pred_train = xgb_best.predict(X_train)

# Display training set evaluation metrics
print('Training Metrics:')
print('R^2:', metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:', 1 - (1 - metrics.r2_score(y_train, y_pred_train)) * (len(y_train) - 1) / (len(y_train) - X_train.shape[1] - 1))
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

# Predict on testing set
y_pred_test = xgb_best.predict(X_test)

# Display testing set evaluation metrics
print()
print('Testing Metrics:')
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

# Display feature importances
xgb_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance Score': xgb_best.feature_importances_
}).sort_values(by='Importance Score', ascending=False)
print()
print('Top 10 important features:')
print(xgb_features.head(10))

## 4.6 LightGBM Regressor

### Training

In [ ]:
# Get var percentage 90 features
data_lgbm = get_var_percentage_features(train_data, 90)

# Split data into features and target
X = data_lgbm.drop(['monthly_rent'], axis=1)
y = data_lgbm['monthly_rent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost hyperparameters
lgbm_param_grid = {
    'metric': ['rmse'],
    'max_depth': [9,10,11,12,13], 
    'bagging_fraction': [0.8, 0.9, 1],
    'feature_fraction': [0.8, 0.9, 1],
    'min_data_in_leaf': [20,50,80],
    'learning_rate': [0.01,0.05,0.1,0.2]
}

# Initialize LightGBM regressor and GridSearchCV
lgbm_reg = lgb.LGBMRegressor(seed = 42, num_iterations = 1200)
lgbm_grid = GridSearchCV(estimator=lgbm_reg, param_grid=lgbm_param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
lgbm_grid = lgbm_grid.fit(X_train, y_train)

# Display best hyperparameters
print()
print('Best hyperparameters:')
print(lgbm_grid.best_params_)

# Get best estimator
lgbm_best = lgbm_grid.best_estimator_

### Evaluating

In [ ]:
# Predict on training set
y_pred_train = lgbm_best.predict(X_train)

# Display training set evaluation metrics
print('Training Metrics:')
print('R^2:', metrics.r2_score(y_train, y_pred_train))
print('Adjusted R^2:', 1 - (1 - metrics.r2_score(y_train, y_pred_train)) * (len(y_train) - 1) / (len(y_train) - X_train.shape[1] - 1))
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train))
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

# Predict on testing set
y_pred_test = lgbm_best.predict(X_test)

# Display testing set evaluation metrics
print()
print('Testing Metrics:')
print('R^2:',metrics.r2_score(y_test, y_pred_test))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_pred_test))
print('MSE:',metrics.mean_squared_error(y_test, y_pred_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

# Display feature importances
lgbm_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance Score': lgbm_best.feature_importances_
}).sort_values(by='Importance Score', ascending=False)
print()
print('Top 10 important features:')
print(lgbm_features.head(10))

# 5. Ensemble Models - Model System Construction

## 5.1 Importing Upstream Work

In [ ]:
train_data = pd.read_csv('preprocessed/train_data.csv')
test_data = pd.read_csv('preprocessed/test_data.csv')
most_contribute_feature = np.load('preprocessed/most_contribute_feature.npy', allow_pickle=True)
feature_contributions = pd.read_csv('preprocessed/feature_contributions.csv')

In [ ]:

def get_var_percentage_features(dataset, percentage, standard=False):
    # Sum up contribution to variance
    total_variance = np.sum(feature_contributions['Contribution to Variance'])

    # Calculate the cumulative contribution to variance in percentage
    feature_contributions['Cumulative Contribution'] = feature_contributions[
                                                           'Contribution to Variance'].cumsum() / total_variance * 100

    # Find the number of features where the cumulative contribution is just over 90%
    k = feature_contributions[feature_contributions['Cumulative Contribution'] <= percentage].shape[0] + 1

    return get_top_k_features(dataset, k, standard)


def get_top_k_features(dataset, k, standard=False):
    """
    Returns a DataFrame with the top-k most contributing features from train_data
    and the target feature 'monthly_rent'.

    Parameters:
    - k (int): Number of top features to select.
    - standard (Boolean): Whether to standardized or not

    Returns:
    - DataFrame: A DataFrame with the top-k features and the target feature.
    """

    # Top-K
    top_k_features = list(most_contribute_feature[:k])
    # Target
    top_k_features.append('monthly_rent')

    if standard:
        # Separate the target feature
        target_feature = dataset['monthly_rent'].copy()

        # Remove the target feature for standardization
        train_data_without_target = dataset.drop(columns=['monthly_rent'])

        train_data_standardized = StandardScaler().fit_transform(train_data_without_target)

        # Convert the array back to a DataFrame
        train_data_standardized_df = pd.DataFrame(train_data_standardized,
                                                  columns=train_data_without_target.columns)

        # Reset indices and add the target feature back to the standardized DataFrame
        train_data_standardized_df = train_data_standardized_df.reset_index(drop=True)
        target_feature = target_feature.reset_index(drop=True)
        train_data_standardized_df['monthly_rent'] = target_feature

    return dataset[top_k_features].copy()


In [ ]:
train_data = get_var_percentage_features(train_data, 90)
test_data = get_var_percentage_features(test_data, 90)
train_data_rent = train_data.loc[:, ['monthly_rent']]
train_data_rent.head()

In [ ]:
# import feature engineering
feature_after_engineering = train_data.columns.tolist()
feature_after_engineering = [i for i in train_data.columns.tolist() if i != 'monthly_rent']

print([i for i in feature_after_engineering])
print(len([i for i in feature_after_engineering]))
train_data_ensemble = train_data.copy().drop(columns=['monthly_rent'])
test_data_ensemble = test_data.copy().drop(columns=['monthly_rent'])
train_data_rent = train_data.loc[:, ['monthly_rent']]

In [ ]:
train_data_ensemble = train_data_ensemble.loc[:, feature_after_engineering].copy()
test_data_ensemble = test_data_ensemble.loc[:, feature_after_engineering].copy()
train_data_final = train_data_ensemble

train_data_ensemble.shape, test_data_ensemble.shape

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def rmsle_cross_val(X_train, y_train, n_folds, model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse = np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv=kf))
    return rmse


In [ ]:
train_data_ensemble.head()

In [ ]:
test_data_ensemble.head()

In [ ]:
train_data_rent.head()

## 5.2 Model Training

### 5.2.1 The Construction of the First Layer of Models

In [ ]:
# # LightGBM model
# param_grid = {
#     'num_leaves': [20, 30, 35, 40],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_bin': [150, 200, 250],
#     'n_estimators': [1000, 1500, 2000],
# }

# model_lgb = lgb.LGBMRegressor(objective='regression', random_state=42)
# grid_search = GridSearchCV(estimator=model_lgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
# grid_search.fit(train_data_ensemble, np.log(train_data_rent))
# best_params = grid_search.best_params_
# print("Best params：", best_params)
# best_model = grid_search.best_estimator_
# print("Best model：", best_model)
# model_lgb = best_model

# params：{'learning_rate': 0.05, 'max_bin': 150, 'n_estimators': 1000, 'num_leaves': 20}
# Best model： LGBMRegressor(learning_rate=0.05, max_bin=150, n_estimators=1000, num_leaves=20,
#               objective='regression', random_state=42)

model_lgb = lgb.LGBMRegressor(objective='regression',
                              num_leaves=20, learning_rate=0.05,
                              max_bin=150, n_estimators=1000,
                              random_state=42)
score_lgb = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_lgb)
print("LGB score: {:.4f}\n".format(score_lgb.mean()))

model_lgb.fit(train_data_ensemble.values, train_data_rent.values)
model_lgb_pred = model_lgb.predict(test_data_ensemble.values)

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_lgb_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/lgb_rmsle_' + str(float(score_lgb.mean()))[:7] + '.csv', index=False)

In [ ]:
# # XGB model
# param_grid = {
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [6, 8, 10],
#     'min_child_weight': [1, 5, 10],
#     'n_estimators': [1000, 2000, 3000],
#     'colsample_bytree': [0.7, 0.8, 0.9],
#     'reg_alpha': [0.1, 0.5, 1.0],
#     'reg_lambda': [0.5, 1.0, 1.5],
#     'subsample': [0.8, 0.9, 1.0]
# }

# model_xgb = xgb.XGBRegressor(random_state=42, nthread=-1)
# grid_search = GridSearchCV(estimator=model_xgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
# grid_search.fit(train_data_ensemble, np.log(train_data_rent))
# best_params = grid_search.best_params_
# print("Best params：", best_params)
# best_model = grid_search.best_estimator_
# print("Best model：", best_model)
# model_xgb = best_model

model_xgb = xgb.XGBRegressor(learning_rate=0.01, gamma=0,
                             max_depth=5, min_child_weight=1,
                             n_estimators=6000, colsample_bytree=0.6,
                             reg_alpha=0.5, reg_lambda=1.0,
                             subsample=0.7, random_state=42, nthread=-1)
score_xgb = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_xgb)
print("XGB rmsle mean: {:.4f}\n".format(score_xgb.mean()))

model_xgb.fit(train_data_ensemble.values, train_data_rent.values)
model_xgb_pred = model_xgb.predict(test_data_ensemble.values)

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_xgb_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/xgb_rmsle_' + str(float(score_xgb.mean()))[:7] + '.csv', index=False)

In [ ]:
# # GBoost model
model_GBoost = GradientBoostingRegressor(n_estimators=7000, learning_rate=0.1,
                                         max_depth=5, max_features='sqrt',
                                         min_samples_leaf=15, min_samples_split=10,
                                         loss='huber', random_state=42)
score_GBoost = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_GBoost)
print("GradientBoosting rmsle mean: {:.4f}\n".format(score_GBoost.mean()))

model_GBoost.fit(train_data_ensemble.values, train_data_rent.values)
model_gb_pred = model_GBoost.predict(test_data_ensemble.values)

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_gb_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/gb_rmsle_' + str(float(score_GBoost.mean()))[:7] + '.csv', index=False)

In [ ]:
# # Random Forest model
# train_data_final = train_data_ensemble[feature_after_engineering].copy()
# model_rf = RandomForestRegressor(oob_score=True, n_jobs=-1)
# params ={
#     'n_estimators': [1200, 1250,1280],
#     'min_samples_leaf': [1],
#     'max_features': [0.5],
#     'max_depth': [35],
#     'min_samples_split': [2]
# }
# best_score = 0
# for g in ParameterGrid(params):
#     model_rf.set_params(**g)
#     model_rf.fit(train_data_final, train_data_rent)
#     if model_rf.oob_score_ > best_score:
#         score_rf = rmsle_cross_val(train_data_final, train_data_rent, 5, model_rf)
#         best_score = model_rf.oob_score_
#         best_grid = g
#         print('best oob:', best_score, best_grid, 'score:', score_rf.mean())

model_rf = RandomForestRegressor(n_jobs=-1, n_estimators=1250, oob_score=True, max_depth=35, min_samples_leaf=1,
                                 min_samples_split=2, max_features=0.5)
score_rf = rmsle_cross_val(train_data_final, train_data_rent, 5, model_rf)
print("Random Forest rmsle mean: {:.4f}\n".format(score_rf.mean()))

model_rf.fit(train_data_ensemble.values, train_data_rent.values)
model_rf_pred = model_rf.predict(test_data_ensemble[feature_after_engineering])

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_rf_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/rf_rmsle_' + str(float(score_rf.mean()))[:7] + '.csv', index=False)

### 5.2.2 The Construction of the Second Layer - Stacking

In [ ]:
# # stacking model
model_stack = StackingCVRegressor(regressors=(model_xgb, model_lgb),
                                  meta_regressor=model_rf,
                                  use_features_in_secondary=True)
score_stack = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_stack)
print("Stacked rmsle mean: {:.4f}\n".format(score_stack.mean()))

model_stack.fit(train_data_ensemble.values, train_data_rent.values)
model_stack_pred = model_stack.predict(test_data_ensemble.values)

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_stack_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/stacked_rmsle_' + str(float(score_stack.mean()))[:7] + '.csv', index=False)

### 5.2.3 The Construction of the Third Layer - Blending

In [ ]:
# weight calculator
denominator = (1 - score_lgb.mean()) + (1 - score_xgb.mean()) + (1 - score_rf.mean()) + (1 - score_stack.mean())

In [ ]:
# # Blend models/Weighting
def blended_predictions(train_x):
    return (((1 - score_lgb.mean()) / denominator * model_xgb.predict(train_x)) + \
            ((1 - score_xgb.mean()) / denominator * model_lgb.predict(train_x)) + \
            ((1 - score_rf.mean()) / denominator * model_rf.predict(train_x)) + \
            ((1 - score_stack.mean()) / denominator * model_stack.predict(train_x)))


model_blend_pred = blended_predictions(train_data_ensemble.values)
score_blend = rmse(np.log(train_data_rent.values), np.log(model_blend_pred))
print(score_blend)
model_blended_pred = blended_predictions(test_data_ensemble.values)

df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_blended_pred), columns=['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
submission.to_csv('submission/blended_rmsle_' + str(score_blend)[:7] + '.csv', index=False)